In [1]:
import pandas as pd

In [2]:
languages = pd.read_csv('languages_and_dialects_geo.csv')

In [3]:
print(languages.describe(include='all'))

       glottocode        name isocodes    level macroarea     latitude  \
count       22111       22111     8121    22111     22019  8849.000000   
unique      22111       22111     8121        2         6          NaN   
top      3adt1234  3Ad-Tekles      aiw  dialect   Eurasia          NaN   
freq            1           1        1    13507      7011          NaN   
mean          NaN         NaN      NaN      NaN       NaN     8.935550   
std           NaN         NaN      NaN      NaN       NaN    19.603034   
min           NaN         NaN      NaN      NaN       NaN   -55.274800   
25%           NaN         NaN      NaN      NaN       NaN    -5.086930   
50%           NaN         NaN      NaN      NaN       NaN     6.842170   
75%           NaN         NaN      NaN      NaN       NaN    21.572100   
max           NaN         NaN      NaN      NaN       NaN    73.135400   

          longitude  
count   8849.000000  
unique          NaN  
top             NaN  
freq            NaN  
m

In [4]:
print(f"The database has {len(languages)} entries on languiges and databases.")

The database has 22111 entries on languiges and databases.


In [5]:
languages['macroarea'].unique()

array(['Africa', 'Papunesia', 'Eurasia', 'South America', 'North America',
       'Australia', nan], dtype=object)